In [1]:
!pip install gradio
!pip install requests beautifulsoup4
!pip install firebase
!pip install plotly
!pip install paho-mqtt
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.2 MB/s eta 0:00:00


In [2]:
import gradio as gr
import pandas as pd
from urllib.parse import urljoin, urldefrag
from collections import Counter
import tempfile # Import tempfile module

In [3]:
import requests
from bs4 import BeautifulSoup
import re
from nltk.stem import PorterStemmer

def fetch_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print("Error fetching page:", e)
    return None


In [4]:
def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text().lower())
    for word in words:
        index[word] = index.get(word, 0) + 1
    return index


In [5]:
def remove_stop_words(index):
    stop_words = {
        'a', 'the', 'and', 'or', 'but', 'if', 'while', 'in', 'on', 'at', 'by', 'for',
        'with', 'of', 'to', 'from', 'about', 'as', 'is', 'was', 'were', 'be', 'this',
        'that', 'these', 'those', 'it', 'he', 'we', 'they', 'you', 'i', 'me', 'my',
        'your', 'his', 'her', 'their', 'them' , 'an','why', 'are','0','1','2','3','4','5','6','7','8','9'
    }
    return {word: count for word, count in index.items() if word not in stop_words}


In [6]:
def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    stem_map = {}  # maps stemmed words to a set of original words

    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
            stem_map[stemmed_word].add(word)
        else:
            stemmed_index[stemmed_word] = count
            stem_map[stemmed_word] = {word}
    return stemmed_index, stem_map


In [7]:
#choose one of the words that stem from the same word
def auto_select_original_words(stem_map, index):
    selected_words = {}

    for stemmed, originals in stem_map.items():
        chosen = sorted(originals, key=lambda w: len(w))[0]  # shortest word
        selected_words[chosen] = index[stemmed]
    word_list = [word.strip().lower() for word in selected_words.keys()]
    return word_list


In [8]:
def search_engine(url):
    soup = fetch_page(url)
    if soup is None:
        print("Failed to fetch page.")
        return

    index = index_words(soup)
    index = remove_stop_words(index)
    stemmed_index, stem_map = apply_stemming(index)
    wordsToDB = auto_select_original_words(stem_map, stemmed_index)
    # Extract description
    description = None
    meta_tag = soup.find("meta", attrs={"name": "description"})
    if meta_tag:
        description = meta_tag.get("content")

    return stemmed_index, wordsToDB, description


In [9]:
#connection to database
from firebase import firebase

firebase_url = "https://cloudhw2-a0d25-default-rtdb.firebaseio.com/"
fb = firebase.FirebaseApplication(firebase_url, None)

In [10]:
import urllib.request

url = "https://github.com/yanivshtein/Cloud-Course/raw/main/cats.png"
filename = "cats.png"

urllib.request.urlretrieve(url, filename)

('cats.png', <http.client.HTTPMessage at 0x78d3783893d0>)

In [11]:
from collections import Counter, defaultdict

url = 'https://mqtt.org/'
stemmed, wordsToDB, description = search_engine(url)


# Map stems to a representative full word (for display/search)
stem_to_word = {}
for stem, word in zip(stemmed, wordsToDB):
    if stem not in stem_to_word:
        stem_to_word[stem] = word  # keep the first full form seen

# Index dictionaries
term_index = {}     # full_word → [(doc_id, count)]
doc_index = {}      # str(doc_id) → URL

doc_id = 1

# Index main page
if stemmed:
    doc_index[str(doc_id)] = {"url": url, "description": description} # Store as dictionary
    main_counts = Counter(stemmed)
    for stem, count in main_counts.items():
        word = stem_to_word[stem]
        if word not in term_index:
            term_index[word] = []
        term_index[word].append((doc_id, count))
    print(f"✔ Indexed main page: {url} with description: {description}")
else:
    print("No results from main page.")
doc_id += 1

# Crawl and index all subpages
try:
    soup = fetch_page(url)
    raw_links = set()
    for a in soup.find_all("a", href=True):
        full_url = urljoin(url, a['href'])
        full_url, _ = urldefrag(full_url)
        raw_links.add(full_url)
except Exception as e:
    print(f"✖ Failed to parse links from main page: {e}")
    raw_links = set()

# Visit and evaluate subpages
for link in raw_links:
    try:
        stemmed_sub, wordsToDB_sub, description_sub = search_engine(link) # Get description
        if stemmed_sub:
            doc_index[str(doc_id)] = {"url": link, "description": description_sub} # Store as dictionary
            sub_counts = Counter(stemmed_sub)
            for stem in sub_counts:
                if stem in stem_to_word:
                    word = stem_to_word[stem]
                    if word not in term_index:
                        term_index[word] = []
                    term_index[word].append((doc_id, sub_counts[stem]))
            print(f"Scanned {link} with description: {description_sub}")
            doc_id += 1
    except Exception as e:
        print(f"Error visiting {link}: {e}")

# Sort doc lists by descending count for each term
for word in term_index:
    term_index[word].sort(key=lambda x: x[1], reverse=True)

print(term_index)
print(doc_index) # Print the updated doc_index to verify

# Save to Firebase
fb.put('/', 'term', term_index)
fb.put('/', 'docIDs', doc_index) # Save the updated doc_index


✔ Indexed main page: https://mqtt.org/ with description: A lightweight messaging protocol for small sensors and mobile devices, optimized for high-latency or unreliable networks, enabling a Connected World and the Internet of Things
Scanned https://mqtt.org/legal with description: Legal notice
Scanned https://mqtt.org/getting-started/ with description: Useful resources to get you started with MQTT, the standard messaging and data exchange protocol for the Internet of Things (IoT).
Scanned https://mqtt.org/use-cases/ with description: MQTT is the top choice of major companies worldwide for data exchange with constrained devices and server applications.
Scanned https://mqtt.org/mqtt-specification/ with description: Links to documentation on the MQTT specification and APIs. High quality MQTT logo download.
Scanned https://mqtt.org/faq/ with description: Frequently asked questions about MQTT and a dictionary of terms and acronyms.
Scanned https://mqtt.org/software/ with description: A coll

[None,
 {'description': 'A lightweight messaging protocol for small sensors and mobile devices, optimized for high-latency or unreliable networks, enabling a Connected World and the Internet of Things',
  'url': 'https://mqtt.org/'},
 {'description': 'Legal notice', 'url': 'https://mqtt.org/legal'},
 {'description': 'Useful resources to get you started with MQTT, the standard messaging and data exchange protocol for the Internet of Things (IoT).',
  'url': 'https://mqtt.org/getting-started/'},
 {'description': 'MQTT is the top choice of major companies worldwide for data exchange with constrained devices and server applications.',
  'url': 'https://mqtt.org/use-cases/'},
 {'description': 'Links to documentation on the MQTT specification and APIs. High quality MQTT logo download.',
  'url': 'https://mqtt.org/mqtt-specification/'},
 {'description': 'Frequently asked questions about MQTT and a dictionary of terms and acronyms.',
  'url': 'https://mqtt.org/faq/'},
 {'description': 'A colle

In [12]:
import random
import time
from firebase import firebase

# Generate fake indoor sensor data
def generate_indoor_data():
    return {
        "Humidity": round(random.uniform(30.0, 60.0), 2),
        "Temperature": round(random.uniform(20.0, 30.0), 2),
        "Pressure": round(random.uniform(970.0, 1020.0), 2),
        "Distance": round(random.uniform(100.0, 300.0), 2)
    }

# Generate fake outdoor sensor data
def generate_outdoor_data():
    return {
        "Humidity": round(random.uniform(20.0, 70.0), 2),
        "Temperature": round(random.uniform(15.0, 35.0), 2),
        "Pressure": round(random.uniform(960.0, 1010.0), 2),
        "DLIGHT": round(random.uniform(5000.0, 50000.0), 2)
    }

# Save multiple samples with 5-minute timestamp intervals
def save_multiple_samples(indoor_count=500, outdoor_count=500):
    print("Generating indoor and outdoor samples...")

    # Start from current time in milliseconds
    start_time = int(time.time() * 1000)
    interval = 5 * 60 * 1000  # 5 minutes in milliseconds

    for i in range(max(indoor_count, outdoor_count)):
        timestamp = str(start_time + i * interval)

        if i < indoor_count:
            indoor = generate_indoor_data()
            fb.put('/sensors/indoor', timestamp, indoor)

        if i < outdoor_count:
            outdoor = generate_outdoor_data()
            fb.put('/sensors/outdoor', timestamp, outdoor)

    print("All samples uploaded to Firebase.")

# Example call:
#save_multiple_samples(indoor_count=250, outdoor_count=250)


In [13]:
#generating real data
import paho.mqtt.client as mqtt
import json
import time
# Containers for data
indoor = []
outdoor = []

# Callback when connected to broker
def on_connect(client, userdata, flags, rc):
    print(" Connected with result code " + str(rc))
    client.subscribe("braude/D106/indoor")
    client.subscribe("braude/D106/outdoor")

# Callback when message is received
def on_message(client, userdata, msg):
    print(f"\n Topic: {msg.topic}")
    try:
        payload = json.loads(msg.payload.decode())
        print(json.dumps(payload, indent=3))

        # Save to appropriate variable
        if msg.topic.endswith("indoor"):
            indoor.append(payload)
        elif msg.topic.endswith("outdoor"):
            outdoor.append(payload)

    except json.JSONDecodeError:
        print("Received non-JSON message:", msg.payload.decode())

"""
# Set up MQTT client
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# Connect and collect data
client.connect("test.mosquitto.org", 1883, 60)
client.loop_start()
time.sleep(5)
client.loop_stop()

"""
# Upload to Firebase
timestamp = int(time.time())

#if indoor_data:
    #FBconn.put('/sensors/indoor', timestamp, indoor)
#if outdoor_data:
    #FBconn.put('/sensors/outdoor', timestamp, outdoor)

In [14]:
def get_worker_data():
    #create workers to DB
    workers_data = [
            {
                "name": "Alice Johnson",
                "role": "Software Engineer",
                "department": "Research & Development"
            },
            {
                "name": "Bob Smith",
                "role": "Senior Technician",
                "department": "Maintenance"
            },
            {
                "name": "Charlie Brown",
                "role": "Financial Analyst",
                "department": "Finance"
            },
            {
                "name": "Diana Prince",
                "role": "Project Manager",
                "department": "Operations"
            },
            {
                "name": "Eve Adams",
                "role": "HR Specialist",
                "department": "Human Resources"
            }
        ]
    return workers_data

def save_worker_data_to_FB():
  workers = get_worker_data()
  for worker in workers:
    fb.put('/workers', worker["name"], worker)

#save_worker_data_to_FB()

In [15]:
#get workers from DB
def get_worker_data_for_db():
   return fb.get('/workers', None)

In [16]:
# Function to get workers as a Pandas DataFrame, using the database-friendly data
def get_workers_df():
    """
    Converts the list of worker dictionaries into a Pandas DataFrame
    with capitalized column names (Name, Role, Department).
    Handles cases where raw_workers might be a dictionary of dictionaries
    (e.g., {'WorkerName': {'key': 'value'}}).
    """
    raw_workers = get_worker_data_for_db()

    # Check if raw_workers is a dictionary (like the user's example)
    # and transform it into a list of dictionaries if it is.
    if isinstance(raw_workers, dict):
        processed_workers = []
        for worker_name, details in raw_workers.items():
            # Create a copy to avoid modifying the original dictionary if it's passed by reference
            worker_dict = details.copy()
            # Ensure the 'name' key is present, using the outer key if not explicitly in details
            if 'name' not in worker_dict:
                worker_dict['name'] = worker_name
            processed_workers.append(worker_dict)
        df = pd.DataFrame(processed_workers)
    else: # Assume it's already a list of dictionaries (the intended format)
        df = pd.DataFrame(raw_workers)

    # Explicitly rename columns using a dictionary mapping.
    # This is robust as it only renames columns that exist.
    df = df.rename(columns={
        "name": "Name",
        "role": "Role",
        "department": "Department"
    })
    return df

# Define the Firebase path for storing assignments
ASSIGNMENTS_FIREBASE_PATH = '/assignments'

# Store assignments in-memory (Initialize this with data from Firebase later)
assignments = {}

# Helper function to save assignments to Firebase
def save_assignments_to_firebase(assignments_data):
    """
    Saves the entire assignments dictionary to Firebase.
    """
    try:
        fb.put('/', 'assignments', assignments_data)
        print("Assignments successfully saved to Firebase.")
    except Exception as e:
        print(f"Error saving assignments to Firebase: {e}")

# Helper function to get assignments from Firebase
def get_assignments_from_firebase():
    """
    Retrieves the assignments dictionary from Firebase.
    Returns an empty dictionary if no data is found.
    """
    try:
        assignments_data = fb.get(ASSIGNMENTS_FIREBASE_PATH, None)
        return assignments_data if isinstance(assignments_data, dict) else {}
    except Exception as e:
        print(f"Error fetching assignments from Firebase: {e}")
        return {}

def format_assignments():
    """
    Formats the current in-memory assignments into a Pandas DataFrame.
    """
    rows = []
    for worker, tasks in assignments.items():
        rows.append({"Worker": worker, "Assignments": ", ".join(tasks)})
    return pd.DataFrame(rows) if rows else pd.DataFrame(columns=["Worker", "Assignments"])


def assign_task(worker_name, task_name):
    """
    Assigns a task to a worker and updates the in-memory assignments and Firebase.
    Does NOT return the formatted DataFrame; that's handled by the wrapper function.
    """
    if not task_name.strip():
        # If no task is entered, do nothing and return
        print("No task name provided.")
        return

    # Ensure the worker exists in the assignments dictionary
    if worker_name not in assignments:
        assignments[worker_name] = []

    # Add the task if it's not already assigned to this worker (optional, prevents duplicates)
    task_to_add = task_name.strip()
    if task_to_add not in assignments[worker_name]:
         assignments[worker_name].append(task_to_add)
         print(f"Assigned '{task_to_add}' to {worker_name}")
         # --- Save the updated assignments to Firebase ---
         save_assignments_to_firebase(assignments)
         # -----------------------------------------
    else:
        print(f"'{task_to_add}' is already assigned to {worker_name}")


def remove_task(worker_name, task_name):
    """
    Removes a task from a worker's assignments and updates the in-memory assignments and Firebase.
    Does NOT return the formatted DataFrame; that's handled by the wrapper function.
    """
    task_to_remove = task_name.strip()

    if worker_name in assignments and task_to_remove in assignments[worker_name]:
        assignments[worker_name].remove(task_to_remove)
        print(f"Removed '{task_to_remove}' from {worker_name}")

        # If no tasks left for the worker, remove the worker from assignments (optional)
        if not assignments[worker_name]:
            del assignments[worker_name]
            print(f"Removed {worker_name} as they have no remaining tasks.")

        # --- Save the updated assignments to Firebase ---
        save_assignments_to_firebase(assignments)
        # -----------------------------------------

    else:
        print(f"Task '{task_to_remove}' not found for {worker_name} or worker does not exist.")


# functions to handle multiple outputs for Gradio events
def assign_task_and_update_dropdown(worker_name, task_name):
    """
    Assigns a task, updates the assignment table, and returns the updated
    choices for the specific worker's task removal dropdown.
    """
    # Call the core assignment logic which updates 'assignments' and saves to Firebase
    # We don't need to capture the return of assign_task here as it's just the DataFrame
    # The side effect is updating the global 'assignments' dict and Firebase
    assign_task(worker_name, task_name)

    # Now, get the updated DataFrame *after* the assignment
    updated_assignments_df = format_assignments()

    # Get the current tasks for this worker to update their dropdown
    worker_tasks = assignments.get(worker_name, [])

    # Return the updated assignment table and the new dropdown state using gr.update
    # The order of outputs MUST match the components listed in the outputs of the click event
    return updated_assignments_df, gr.update(choices=worker_tasks, value=None)

def remove_task_and_update_dropdown(worker_name, task_name_to_remove):
    """
    Removes a task, updates the assignment table, and returns the updated
    choices for the specific worker's task removal dropdown.
    """
    # Call the core removal logic which updates 'assignments' and saves to Firebase
    # The side effect is updating the global 'assignments' dict and Firebase
    remove_task(worker_name, task_name_to_remove)

    # Now, get the updated DataFrame *after* the removal
    updated_assignments_df = format_assignments()

    # Get the current tasks for this worker to update their dropdown
    worker_tasks = assignments.get(worker_name, [])

    # Return the updated assignment table and the new dropdown state using gr.update
    # The order of outputs MUST match the components listed in the outputs of the click event
    return updated_assignments_df, gr.update(choices=worker_tasks, value=None)


# Function to build the UI for the Worker Assignment Manager tab
def build_manager_tab():
    global assignments
    assignments = get_assignments_from_firebase()
    print("Loaded assignments from Firebase:", assignments)

    df = get_workers_df() # Get the list of workers

    # Lists to hold the dropdown components for each worker
    remove_dropdowns = []

    with gr.Column(elem_classes="manager-tab-container gr-box"):
        gr.Markdown("<h2 class='text-2xl font-bold text-center text-blue-700 mb-6'>Worker Assignment Manager</h2>")
        gr.Markdown("<p class='text-gray-600 text-center mb-4'>Assign and remove tasks for your workforce. Data is saved to Firebase.</p>")

        # Dataframe to display current assignments
        assignment_table = gr.Dataframe(
            headers=["Worker", "Assignments"],
            datatype=["str", "str"],
            interactive=False,
            row_count=(3, "dynamic"),
            col_count=(2, "fixed"),
            value=format_assignments(),
            elem_classes="assignment-table"
        )

        gr.Markdown("<h3 class='text-xl font-semibold text-gray-800 mt-8 mb-4'>Workers & Task Management</h3>")
        # Loop through each worker to create input fields and buttons
        for idx, row in df.iterrows():
            worker_name = row["Name"]
            # Get tasks for this worker to populate the initial dropdown
            initial_tasks = assignments.get(worker_name, [])

            with gr.Row(elem_classes="worker-row"):
                # Display worker information
                gr.Textbox(value=worker_name, interactive=False, label="Name", elem_classes="worker-info-box")
                gr.Textbox(value=row["Role"], interactive=False, label="Role", elem_classes="worker-info-box")
                gr.Textbox(value=row["Department"], interactive=False, label="Department", elem_classes="worker-info-box")

                with gr.Column(scale=2): # Column for task input and remove dropdown
                    task_input = gr.Textbox(placeholder="Enter new task...", label="New Task", elem_classes="task-input")
                    # REPLACE Textbox with Dropdown for removing tasks
                    remove_dropdown = gr.Dropdown(
                        label="Select Task to Remove",
                        choices=initial_tasks, # Populate with initial tasks
                        value=None, # No task selected by default
                        interactive=True,
                        elem_classes="remove-dropdown"
                    )
                    # Append the remove_dropdown component to the list
                    remove_dropdowns.append(remove_dropdown)


                with gr.Column(scale=1, min_width=150): # Column for action buttons
                    assign_button = gr.Button("Assign Task", elem_classes="action-button assign-button")
                    remove_button = gr.Button("Remove Selected Task", elem_classes="action-button remove-button")


                # Connect assign button to assign_task_and_update_dropdown function
                # Outputs: updated table, AND the remove dropdown for this worker
                assign_button.click(
                  fn=assign_task_and_update_dropdown,
                  inputs=[gr.State(worker_name), task_input],
                  # The outputs are the assignment table AND the specific remove_dropdown for this worker
                  outputs=[assignment_table, remove_dropdown]
                )

                # Connect remove button to remove_task_and_update_dropdown function
                # Inputs: worker_name, AND the selected value from the remove dropdown for this worker
                # Outputs: updated table, AND the remove dropdown for this worker
                remove_button.click(
                  fn=remove_task_and_update_dropdown,
                  inputs=[gr.State(worker_name), remove_dropdown], # Use remove_dropdown as input
                   # The outputs are the assignment table AND the specific remove_dropdown for this worker
                   outputs=[assignment_table, remove_dropdown]
               )

    # We need to return the assignment_table and the list of remove_dropdowns
    # for the tab.select event if we were to implement a refresh on tab switch.
    # For now, we'll just return the table as initially planned, but keep the dropdowns
    # list in case it's needed for future enhancements.
    return assignment_table # Returning just the table is sufficient for current needs


In [17]:
# --- Search Tab Functions ---

# Load terms and doc index from Firebase
term_index = fb.get('/', 'term') or {}
doc_index = fb.get('/', 'docIDs') or []
known_terms = sorted(term_index.keys())

# Function to search for documents based on a query
def search_term(query):
    if not query:
        return "Enter one or more terms to search."

    # Split query into individual terms and convert to lowercase
    terms = [term.strip().lower() for term in query.split() if term.strip()]
    matched_doc_ids = set()

    # Load latest indices from Firebase
    term_index = fb.get('/', 'term') or {}
    # Assume doc_index is a list when retrieved from Firebase
    doc_index = fb.get('/', 'docIDs') or [] # Default to empty list if not found or not a list


    # Find all document IDs that contain any of the query terms
    for term in terms:
        if term in term_index:
            # term_index[term] is a list of (doc_id, count) tuples
            matched_doc_ids.update(str(doc_id) for doc_id, _ in term_index[term]) # Ensure doc_id is string


    if not matched_doc_ids:
        return "No results found."

    # Calculate a score for each document based on term frequency
    doc_id_scores = {}
    for term in terms:
        # Iterate through the (doc_id, count) tuples for the term
        for doc_id, count in term_index.get(term, []):
             # Use the string representation of doc_id for consistency with doc_index keys
             doc_id_scores[str(doc_id)] = doc_id_scores.get(str(doc_id), 0) + count


    # Sort document IDs by their score in descending order
    sorted_doc_ids = sorted(doc_id_scores, key=doc_id_scores.get, reverse=True)

    # Format the results as a list of markdown links with descriptions
    links = []
    for doc_id_str in sorted_doc_ids: # Iterate through string doc_ids
        try:
            # Convert the string doc_id back to an integer index for list access
            doc_id_int = int(doc_id_str)
            # Access the document info from the list using the integer index
            # Check if the index is within the bounds of the list
            if 0 <= doc_id_int < len(doc_index):
                doc_info = doc_index[doc_id_int]
                # Ensure the retrieved item is a dictionary before accessing keys
                if isinstance(doc_info, dict):
                    url = doc_info.get("url")
                    description = doc_info.get("description")
                    if url:
                        links.append(f"- [{url}]({url})")
                        if description:
                            links.append(f"\n{description}")
                    else:
                        links.append(f"- Doc {doc_id_str} (URL missing in index)")
                else:
                    links.append(f"- Doc {doc_id_str} (Info not a dictionary in index)")
            else:
                links.append(f"- Doc {doc_id_str} (Index out of bounds in index)")
        except ValueError:
             links.append(f"- Doc {doc_id_str} (Invalid doc ID format in index)")
        except IndexError:
             links.append(f"- Doc {doc_id_str} (Index error accessing list in index)")


    return "\n".join(links)

# Function to build the UI for the Document Search tab
def build_search_tab():
    with gr.Column(elem_classes="search-tab-container gr-box"):
      gr.Image(
        value="/content/cats.png",
        show_download_button=False,
        show_label=False,
        show_fullscreen_button=False,
        interactive=False,
        width=400
      )

    with gr.Row(elem_classes="search-container-row no-margin"):
        # Wrap dropdown in a column with a max width to center it cleanly
        with gr.Column(scale=1, min_width=300, elem_classes="search-container-inner"):
            search_dropdown = gr.Dropdown(
                label="Select or Type Search Term(s)",
                choices=known_terms,
                interactive=True,
                allow_custom_value=True,
                elem_classes="search-dropdown"
            )

    search_button = gr.Button("Search Documents", elem_classes="search-button")
    search_output = gr.Markdown(elem_classes="search-output")

    search_button.click(
        fn=search_term,
        inputs=search_dropdown,
        outputs=search_output
    )


    return search_dropdown, search_button, search_output

In [18]:
import plotly.express as px
import xlsxwriter # Make sure you have this library installed: pip install xlsxwriter

# Define thresholds for each metric
THRESHOLDS = {
    "Temperature": 30.0,
    "Humidity": 60.0,
    "Pressure": 1010.0,
    "Distance": 250.0,
    "DLIGHT": 40000.0
}

# Define which metrics belong to which sensor type
SENSOR_METRICS = {
    "indoor": ["Temperature", "Humidity", "Pressure", "Distance"],
    "outdoor": ["Temperature", "Humidity", "Pressure", "DLIGHT"]
}

# Function to fetch sensor data from the mock Firebase
def fetch_sensor_data(sensor_type="indoor"):
    """Fetch and return a sorted DataFrame of sensor readings."""
    path = f'/sensors/{sensor_type}'
    raw_data = fb.get(path, None)

    if not raw_data:
        return pd.DataFrame() # Return empty DataFrame if no data

    records = []
    for timestamp, readings in raw_data.items():
        try:
            ts = int(timestamp)
            readings["timestamp"] = pd.to_datetime(ts, unit='ms') # Convert timestamp to datetime object
            records.append(readings)
        except ValueError: # Handle cases where timestamp might not be a valid integer
            print(f"Skipping invalid timestamp: {timestamp}")
            continue

    df = pd.DataFrame(records)
    return df.sort_values("timestamp") # Sort data by timestamp

# function to export plot data to Excel
import pandas as pd
import tempfile
import xlsxwriter # Make sure you have this library installed: pip install xlsxwriter

def export_plot_data_to_excel(sensor_type, metric):
    df = fetch_sensor_data(sensor_type)

    # Check if both the metric and timestamp columns exist in the DataFrame and the metric column has data
    if metric in df.columns and 'timestamp' in df.columns and not df[metric].empty:
        # Select both the 'timestamp' and the specified 'metric' columns
        export_df = df[["timestamp", metric]].copy()

        with tempfile.NamedTemporaryFile(suffix=".xlsx", delete=False) as tmp_file:
            temp_file_path = tmp_file.name

            # Create a Pandas Excel writer using XlsxWriter as the engine.
            with pd.ExcelWriter(temp_file_path, engine='xlsxwriter') as writer:
                # Convert the DataFrame to an XlsxWriter Excel object.
                export_df.to_excel(writer, sheet_name='Sensor Data', index=False)

                # Get the xlsxwriter workbook and worksheet objects.
                workbook  = writer.book
                worksheet = writer.sheets['Sensor Data']

                # Add a format for the timestamp column to ensure it's treated as a date/time
                # You can adjust the format string as needed.
                timestamp_format = workbook.add_format({'num_format': 'yyyy-mm-dd hh:mm:ss'})

                # Set the column width and format for the timestamp column (column A, index 0).
                # The width is in characters. You might need to adjust 20 based on your timestamps.
                worksheet.set_column('A:A', 20, timestamp_format)

                # You can optionally set the width of the metric column as well
                # worksheet.set_column('B:B', 15) # Example: set width of column B to 15

            print(f"Data for {sensor_type} - {metric} exported to temporary file: {temp_file_path} with column formatting.")

        # Return a Gradio update for the File component
        # Set the value to the file path and explicitly make it visible
        return gr.update(value=temp_file_path, visible=True)
    else:
        print(f"No data available for {sensor_type} - {metric} to export or 'timestamp' column missing.")
        # Return a Gradio update to hide the File component and clear its value
        return gr.update(value=None, visible=False)



def generate_all_plots(sensor_type):
    df = fetch_sensor_data(sensor_type)

    plot_updates = []
    container_updates = []
    # Updates for export button visibility
    export_button_updates = []
    # Updates for gr.File component visibility and value (clear previous downloads)
    export_file_updates = []

    # Get the list of expected metrics for the selected sensor type
    expected_metrics = SENSOR_METRICS.get(sensor_type, [])

    # Iterate over ALL possible metrics defined in THRESHOLDS.
    # This ensures the output list order and count match build_statistics_tab.
    for col in THRESHOLDS.keys():
        # Check if the current metric is expected for this sensor type AND
        # if the column exists in the DataFrame and has data.
        if col in expected_metrics and col in df.columns and not df[col].empty:
            fig = px.line(df, x="timestamp", y=col, title=f"{col} over time")

            # --- Plotly Layout Improvements ---
            fig.update_layout(
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                font_color="#333333",
                title_font_size=18,
                title_font_color="#2d3748",
                xaxis_title_font_color="#555555",
                yaxis_title_font_color="#555555",
                xaxis_tickfont_color="#555555",
                yaxis_tickfont_color="#555555",
                legend_font_color="#555555",
                margin=dict(l=40, r=40, t=40, b=40)
            )
            # Add threshold line if defined
            threshold = THRESHOLDS.get(col)
            if threshold is not None:
                fig.add_hline(
                    y=threshold,
                    line_dash="dash",
                    line_color="red",
                    annotation_text=f"Threshold: {threshold}",
                    annotation_position="top left",
                    annotation_font_color="red"
                )

            # Append gr.update to show this plot and its container
            plot_updates.append(gr.update(value=fig, visible=True))
            container_updates.append(gr.update(visible=True))
            # Make the export button visible and hide the file component (clearing previous download)
            export_button_updates.append(gr.update(visible=True))
            export_file_updates.append(gr.update(value=None, visible=False))
        else:
            # If the metric is not expected for this sensor type OR data is missing,
            # hide this plot and its container, and also the export button and file component.
            print(f"DEBUG: Metric {col} is not expected for {sensor_type} or data is missing. Plot, container, button, and file will be hidden.")
            plot_updates.append(gr.update(value=None, visible=False))
            container_updates.append(gr.update(visible=False))
            export_button_updates.append(gr.update(visible=False))
            export_file_updates.append(gr.update(value=None, visible=False))


    # Return the combined updates for plots, containers, buttons, and files
    # The order MUST match the order of components in the outputs list of the .change and .select methods
    return plot_updates + container_updates + export_button_updates + export_file_updates


# Function to build the UI for the Sensor Statistics tab
def build_statistics_tab():
    plot_components = []
    plot_containers = []
    export_buttons = []
    export_files = []

    with gr.Column(elem_classes="stats-tab-container gr-box"):
        gr.Markdown("<h2 class='text-2xl font-bold text-center text-purple-700 mb-6'>📊 Sensor Statistics and Reports</h2>")
        gr.Markdown("<p class='text-gray-600 text-center mb-4'>Select a sensor type to view historical data and identify trends with predefined thresholds.</p>")

        with gr.Row(elem_classes="sensor-type-selector"):
            stats_sensor_type_dropdown = gr.Dropdown(
                list(SENSOR_METRICS.keys()), # Use keys from SENSOR_METRICS for dropdown choices
                label="Select Sensor Type",
                value="indoor", # Default value
                scale=1,
                interactive=True,
                elem_classes="sensor-dropdown"
            )

        # Use a simple Column to stack plot cards vertically
        with gr.Column(): # You can add elem_classes here if you want a container around the stacked plots
            # Create components for *each* possible metric in THRESHOLDS
            for metric in THRESHOLDS.keys():
                # Create the container column for each plot card
                # Remove min_width if it's preventing full width
                with gr.Column(elem_classes="plot-card") as plot_card_container:
                    # Create the plot component inside the container
                    plot_comp = gr.Plot(label=f"{metric} Data", visible=False, show_label=False, elem_classes="sensor-plot")
                    # Append the plot component to the list
                    plot_components.append(plot_comp)
                    # Append the container to the list
                    plot_containers.append(plot_card_container)

                    # Add export button and file component within the plot card
                    with gr.Row():
                        export_btn = gr.Button(f"Export {metric} Data to Excel", visible=False, size="sm", elem_classes="export-button")
                        export_buttons.append(export_btn)

                        export_file_comp = gr.File(label="Download Excel", visible=False, file_count="single")
                        export_files.append(export_file_comp)

                    export_btn.click(
                        fn=export_plot_data_to_excel,
                        inputs=[stats_sensor_type_dropdown, gr.State(metric)],
                        outputs=export_file_comp
                    )

        stats_sensor_type_dropdown.change(
            fn=generate_all_plots,
            inputs=stats_sensor_type_dropdown,
            outputs=plot_components + plot_containers + export_buttons + export_files
        )

    return stats_sensor_type_dropdown, plot_components, plot_containers, export_buttons, export_files


In [19]:
# Define the Firebase path for storing tips
TIPS_FIREBASE_PATH = '/tips'

# --- Knowledge Hub Tab Functions ---

def format_tips_to_df(tips_list):
    """
    Formats a list of tip dictionaries into a Pandas DataFrame for display.
    Each dictionary should have 'uploader' and 'tip' keys.
    """
    if not tips_list:
        # Return an empty DataFrame with the correct column names
        return pd.DataFrame(columns=["Uploader", "Tip"])
    else:
        # Ensure each item is a dictionary with expected keys, provide defaults if needed
        processed_tips = []
        for item in tips_list:
            if isinstance(item, dict):
                processed_tips.append({
                    "Uploader": item.get("uploader", "Unknown"),
                    "Tip": item.get("tip", "Invalid Tip Data")
                })
            else:
                 # Handle old format tips if they exist initially (optional but robust)
                 processed_tips.append({
                    "Uploader": "Unknown (Old Format)",
                    "Tip": str(item) # Convert old string tip to string
                 })


        return pd.DataFrame(processed_tips)

def get_tips_from_firebase():
    """
    Retrieves the list of tip dictionaries from Firebase.
    Returns an empty list if no data is found.
    """
    try:
        tips_data = fb.get(TIPS_FIREBASE_PATH, None)
        print(tips_data)
        # Firebase might return None, a dictionary (if keys are not integers), or a list
        # We expect a list of dictionaries.
        if isinstance(tips_data, list):
             return tips_data
        elif isinstance(tips_data, dict):
            # If it's a dict (Firebase sometimes does this with non-integer keys),
            # convert values to a list. This assumes the dict values are the tip objects.
            print("Warning: Tips data fetched as dictionary, converting to list.")
            return list(tips_data.values())
        else:
            return [] # Return empty list if None or other unexpected type
    except Exception as e:
        print(f"Error fetching tips from Firebase: {e}")
        return [] # Return empty list on error


def save_tips_to_firebase(tips_list):
    """
    Saves the entire list of tip dictionaries to Firebase.
    """
    print(tips_list)
    try:
        # Use put to overwrite the entire list at the specified path
        fb.put('/', 'tips', tips_list)
        print("Tips successfully saved to Firebase.")
    except Exception as e:
        print(f"Error saving tips to Firebase: {e}")


def add_tip(current_tips, uploader_name, new_tip_text):
    """
    Adds a new tip with uploader name to the list of tips and saves to Firebase.

    Args:
        current_tips (list): The current list of tip dictionaries (from gr.State).
        uploader_name (str): The name of the person adding the tip.
        new_tip_text (str): The tip text entered by the user.

    Returns:
        tuple: (updated_tips_list, tips_dataframe, clear_tip_input, clear_name_input, delete_dropdown_choices)
               - updated_tips_list: The list of tips with the new tip added.
               - tips_dataframe: The DataFrame representation for display.
               - clear_tip_input: gr.update to clear the tip textbox.
               - clear_name_input: gr.update to clear the name textbox.
               - delete_dropdown_choices: gr.update with updated choices for delete dropdown.
    """
    uploader_name = uploader_name.strip() if uploader_name else "Anonymous"
    tip_text = new_tip_text.strip()

    if tip_text: # Only add if the tip text is not empty
        new_tip_entry = {"uploader": uploader_name, "tip": tip_text}
        # Fetch latest from Firebase to prevent overwriting others' tips
        latest_tips = get_tips_from_firebase()
        updated_tips = latest_tips + [new_tip_entry]

        # Save the updated list to Firebase
        save_tips_to_firebase(updated_tips)

        # Return updates
        # The delete dropdown choices need to reflect the current tips
        delete_choices = [f"{tip['uploader']}: {tip['tip'][:50]}..." if len(tip['tip']) > 50 else f"{tip['uploader']}: {tip['tip']}" for tip in updated_tips]

        return (updated_tips,
                format_tips_to_df(updated_tips),
                gr.update(value=""), # Clear tip input
                gr.update(value=""), # Clear name input
                gr.update(choices=delete_choices, value=None)) # Update delete dropdown

    else:
        # If no tip was entered, just return the current state and DataFrame
        # Also update the delete dropdown choices just in case (no change if no tip added)
        delete_choices = [f"{tip['uploader']}: {tip['tip'][:50]}..." if len(tip['tip']) > 50 else f"{tip['uploader']}: {tip['tip']}" for tip in current_tips]
        return (current_tips,
                format_tips_to_df(current_tips),
                gr.update(value=new_tip_text), # Don't clear tip input
                gr.update(value=uploader_name), # Don't clear name input
                gr.update(choices=delete_choices, value=None)) # Update delete dropdown


def delete_tip(current_tips, tip_to_delete_representation):
    """
    Deletes a selected tip from the list and saves to Firebase.

    Args:
        current_tips (list): The current list of tip dictionaries (from gr.State).
        tip_to_delete_representation (str): The string representation of the tip
                                           selected from the delete dropdown.

    Returns:
        tuple: (updated_tips_list, tips_dataframe, clear_delete_dropdown, delete_dropdown_choices)
               - updated_tips_list: The list of tips with the deleted tip removed.
               - tips_dataframe: The DataFrame representation for display.
               - clear_delete_dropdown: gr.update to clear the delete dropdown selection.
               - delete_dropdown_choices: gr.update with updated choices for delete dropdown.
    """
    if not tip_to_delete_representation:
        # No tip selected to delete, return current state
        delete_choices = [f"{tip['uploader']}: {tip['tip'][:50]}..." if len(tip['tip']) > 50 else f"{tip['uploader']}: {tip['tip']}" for tip in current_tips]
        return (current_tips,
                format_tips_to_df(current_tips),
                gr.update(value=None),
                gr.update(choices=delete_choices))


    # Find the tip to delete based on its string representation
    # This assumes the representation uniquely identifies the tip for now.
    # A more robust solution might involve using an index or a unique ID.
    latest_tips = get_tips_from_firebase()
    updated_tips = []
    deleted_count = 0
    for tip in latest_tips:

        tip_representation = f"{tip.get('uploader', 'Unknown')}: {tip.get('tip', 'Invalid Tip Data')[:50]}..." if len(tip.get('tip', '')) > 50 else f"{tip.get('uploader', 'Unknown')}: {tip.get('tip', 'Invalid Tip Data')}"

        if tip_representation == tip_to_delete_representation and deleted_count == 0:
            # Found the tip to delete, skip it. Only delete the first match.
            deleted_count += 1
            print(f"Deleting tip: {tip_to_delete_representation}")
        else:
            updated_tips.append(tip) # Keep tips that are not being deleted

    if deleted_count > 0:
        # Save the updated list to Firebase only if something was deleted
        save_tips_to_firebase(updated_tips)

    # Return updates
    delete_choices = [f"{tip['uploader']}: {tip['tip'][:50]}..." if len(tip['tip']) > 50 else f"{tip['uploader']}: {tip['tip']}" for tip in updated_tips]
    return (updated_tips,
            format_tips_to_df(updated_tips),
            gr.update(value=None), # Clear delete dropdown selection
            gr.update(choices=delete_choices)) # Update delete dropdown choices


def compute_leaderboard(tips: list):
    # Count tips per uploader
    counter = {}
    for tip in tips:
        uploader = tip.get("uploader", "Anonymous")
        counter[uploader] = counter.get(uploader, 0) + 1

    # Sort by tip count (descending)
    sorted_leaderboard = sorted(counter.items(), key=lambda x: x[1], reverse=True)

    # Add medals
    medals = ["🥇", "🥈", "🥉"]
    leaderboard_with_medals = []
    for i, (name, count) in enumerate(sorted_leaderboard):
        medal = medals[i] if i < 3 else ""
        leaderboard_with_medals.append([f"{medal} {name}", count])

    return leaderboard_with_medals


# Function to build the UI for the Knowledge Hub tab
def build_knowledge_hub_tab():
    # --- Get existing tips from Firebase when the tab is built ---
    initial_tips = get_tips_from_firebase()
    # -------------------------------------------------------------

    # Initialize the gr.State component with the tips fetched from Firebase
    tips_state = gr.State(initial_tips)

    with gr.Column(elem_classes="knowledge-hub-container gr-box"):
        gr.Markdown("<h2 class='text-2xl font-bold text-center text-green-700 mb-6'>💡 Knowledge Hub: Tips and Tricks</h2>")
        gr.Markdown("<p class='text-gray-600 text-center mb-4'>Share helpful tips and insights with your colleagues.</p>")

        with gr.Row(): # Use a row for uploader name and tip input
             uploader_name_input = gr.Textbox(
                label="Your Name",
                placeholder="Enter your name...",
                scale=1, # Take up some space, but less than tip input
                elem_classes="tip-input-name"
             )
             # Input area for new tips
             new_tip_input = gr.Textbox(
                label="Add a New Tip or Trick",
                placeholder="Enter your helpful tip here...",
                lines=3,
                scale=2, # Take up more space
                elem_classes="tip-input"
             )


        # Button to add the tip
        add_tip_button = gr.Button("Add Tip", elem_classes="action-button add-tip-button")

        gr.Markdown("<h3 class='text-xl font-semibold text-gray-800 mt-8 mb-4'>Shared Tips</h3>")

        # DataFrame to display the tips, initialized with data from Firebase
        tips_table = gr.DataFrame(
            # Update headers to match the new DataFrame structure
            headers=["Uploader", "Tip"],
            datatype=["str", "str"],
            interactive=False,
            row_count=(5, "dynamic"),
            col_count=(2, "fixed"), # Two columns now
            # Initialize with the tips fetched from Firebase
            value=format_tips_to_df(initial_tips),
            elem_classes="tips-table"
        )

        gr.Markdown("<h3 class='text-xl font-semibold text-gray-800 mt-8 mb-4'>Delete a Tip</h3>")

        with gr.Row(): # Row for delete functionality
             # Dropdown to select a tip to delete
             # Populate choices initially based on fetched tips
             delete_tip_dropdown_choices = [f"{tip['uploader']}: {tip['tip'][:50]}..." if len(tip['tip']) > 50 else f"{tip['uploader']}: {tip['tip']}" for tip in initial_tips]

             delete_tip_dropdown = gr.Dropdown(
                label="Select Tip to Delete",
                choices=delete_tip_dropdown_choices, # Initial choices
                value=None, # No default selection
                interactive=True,
                scale=3,
                elem_classes="delete-tip-dropdown"
             )

             # Button to delete the selected tip
             delete_tip_button = gr.Button("Delete Selected Tip", scale=1, elem_classes="action-button delete-tip-button")


                # --- Leaderboard Section (Always Visible) ---
        with gr.Column():
            gr.Markdown("### 🏆 Leaderboard: Most Tips Submitted")
            leaderboard_table = gr.DataFrame(
                headers=["Uploader", "Tips Count"],
                datatype=["str", "int"],
                interactive=False,
                visible=True,
                value=compute_leaderboard(initial_tips),
                elem_classes="leaderboard-table"
            )



        # Connect the add button click to the add_tip function
        # Update outputs to include the uploader name input and delete dropdown
        add_tip_button.click(
            fn=add_tip,
            inputs=[tips_state, uploader_name_input, new_tip_input],
            outputs=[tips_state, tips_table, new_tip_input, uploader_name_input, delete_tip_dropdown]
        ).then(
            fn=compute_leaderboard,
            inputs=[tips_state],
            outputs=[leaderboard_table]
        )


        # Connect the delete button click to the delete_tip function
        # Update outputs to include the delete dropdown itself (to clear selection)
        delete_tip_button.click(
            fn=delete_tip,
            inputs=[tips_state, delete_tip_dropdown],
            outputs=[tips_state, tips_table, delete_tip_dropdown, delete_tip_dropdown]
        ).then(
            fn=compute_leaderboard,
            inputs=[tips_state],
            outputs=[leaderboard_table]
        )



    # Return components if needed elsewhere (now also includes uploader name input, delete dropdown, delete button)
    return uploader_name_input, new_tip_input, add_tip_button, tips_table, delete_tip_dropdown, delete_tip_button


In [20]:
# --- Gradio App Interface Definition ---
# The main Gradio Blocks interface
with gr.Blocks(theme=gr.themes.Soft(), title="Workforce & Data Dashboard") as demo:
    # Overall application title and description
    gr.Markdown("""
    <h1 class='text-4xl font-extrabold text-center text-gray-900 mb-8'>Workforce & Data Dashboard</h1>
    """)

    # Create tabs for each section of the application
    with gr.Tab("Manage Assignments") as tab_manager:
        manager_output = build_manager_tab() # Build and get components from manager tab

    with gr.Tab("Document Search") as tab_search:
        search_dropdown, search_button, search_output = build_search_tab() # Build and get components from search tab

    with gr.Tab("Sensor Statistics") as tab_stats:
        # Get the dropdown, list of plot components, and list of containers
        stats_sensor_type_dropdown, stats_plots, stats_containers, stats_export_buttons, stats_export_files = build_statistics_tab()
    with gr.Tab("Knowledge Hub") as tab_knowledge:
        uploader_name_input_kh, new_tip_input_kh, add_tip_button_kh, tips_table_kh, delete_tip_dropdown_kh, delete_tip_button_kh = build_knowledge_hub_tab() # Call the function to build the new tab's UI

    # Trigger plot generation when the "Sensor Statistics" tab is selected
    tab_stats.select(
        fn=generate_all_plots,
        inputs=stats_sensor_type_dropdown,
        # Concatenate lists of plots, containers, buttons, and files for outputs
        # ADD stats_export_buttons and stats_export_files HERE
        outputs=stats_plots + stats_containers + stats_export_buttons + stats_export_files
    )

    # Function to refresh the display components (DataFrame and Dropdown)
    def refresh_display_on_select():
        latest_tips = get_tips_from_firebase() # Fetch the latest data from Firebase
        latest_df = format_tips_to_df(latest_tips) # Format it into a DataFrame
        # Create the choices for the delete dropdown from the latest tips
        delete_choices = [f"{tip['uploader']}: {tip['tip'][:50]}..." if len(tip['tip']) > 50 else f"{tip['uploader']}: {tip['tip']}" for tip in latest_tips]

        # Return ONLY the updates for the visible components that need refreshing
        return latest_df, gr.update(choices=delete_choices, value=None)


    tab_knowledge.select(
        fn=refresh_display_on_select,
        # No inputs needed as it fetches directly from Firebase
        inputs=None,
        # Outputs: updated DataFrame, updated delete dropdown
        # These must match the components returned by build_knowledge_hub_tab that you want to update
        outputs=[tips_table_kh, delete_tip_dropdown_kh]
    )
    # REVISED tab_knowledge.select - just update the display components
    def refresh_display_on_select():
        latest_tips = get_tips_from_firebase()
        latest_df = format_tips_to_df(latest_tips)
        delete_choices = [f"{tip['uploader']}: {tip['tip'][:50]}..." if len(tip['tip']) > 50 else f"{tip['uploader']}: {tip['tip']}" for tip in latest_tips]
        # Return only the components that need refreshing in the UI
        return latest_df, gr.update(choices=delete_choices, value=None)


    tab_knowledge.select(
        fn=refresh_display_on_select,
        inputs=None,
        # Outputs: updated DataFrame, updated delete dropdown
        outputs=[tips_table_kh, delete_tip_dropdown_kh]
    )

    # Custom CSS for styling the Gradio components
    demo.css = """
    /* Styling for Gradio 'boxes' (Columns, Rows, etc.) */
.gr-box {
    border-radius: 12px;
    box-shadow: 0 6px 20px rgba(0, 0, 0, 0.1); /* Enhanced shadow for individual sections */
    padding: 28px;
    background-color: #ffffff;
    margin-bottom: 5px;
    border: 1px solid #e2e8f0; /* Subtle border */
}
/* Header styling */
h1, h2, h3 {
    color: #2d3748; /* Darker, more professional text for headers */
    font-family: 'Inter', sans-serif;
}
/* Text alignment and color utilities */
.text-center { text-align: center; }
.text-blue-700 { color: #2563eb; } /* Deeper blue for Manager tab header */
.text-green-700 { color: #059669; } /* Deeper green for Search tab header */
.text-purple-700 { color: #7c3aed; } /* Deeper purple for Statistics tab header */
.text-gray-900 { color: #1a202c; }
.text-gray-700 { color: #4a5568; }
.text-gray-600 { color: #718096; }
/* Font weight utilities */
.font-bold { font-weight: 700; }
.font-semibold { font-weight: 600; }
.font-extrabold { font-weight: 800; }
/* Font size utilities */
.text-2xl { font-size: 1.65rem; } /* Slightly larger */
.text-xl { font-size: 1.35rem; }
.text-lg { font-size: 1.18rem; }
.text-4xl { font-size: 2.5rem; } /* Larger main title */
/* Margin utilities */
.mb-4 { margin-bottom: 1rem; }
.mb-6 { margin-bottom: 1.5rem; }
.mb-8 { margin-bottom: 2rem; }
.mb-10 { margin-bottom: 2.5rem; }
.mt-8 { margin-top: 2rem; }

/* Gradio Tabs Styling */
/* Remove default tab bar styling */
.gradio-tabs > div[role="tablist"] {
    background-color: transparent;
    border-bottom: none;
    border-radius: 0;
    display: flex; /* Arrange buttons in a row */
    justify-content: center; /* Center the buttons */
    gap: 15px; /* Add space between buttons */
    margin-bottom: 20px; /* Space between buttons and content */
}

/* Style individual tab buttons */
.gradio-tabs > div[role="tablist"] button {
    font-weight: 600;
    color: #4a5568;
    padding: 12px 25px; /* Adjust padding for button size */
    transition: background-color 0.3s ease, color 0.3s ease, box-shadow 0.3s ease;
    border: 1px solid #cbd5e0; /* Add a border */
    border-radius: 8px; /* Rounded corners for buttons */
    background-color: #ffffff; /* White background for unselected buttons */
    cursor: pointer;
    box-shadow: 0 2px 5px rgba(0, 0, 0, 0.05);
}

/* Style for the selected tab button */
.gradio-tabs > div[role="tablist"] button.selected {
    background-color: #2563eb; /* Blue background for selected button */
    color: white; /* White text for selected button */
    border-color: #2563eb; /* Blue border for selected button */
    box-shadow: 0 4px 10px rgba(37, 99, 235, 0.3); /* Add a shadow for selected button */
}

/* Hover effect for tab buttons */
.gradio-tabs > div[role="tablist"] button:not(.selected):hover {
    background-color: #f0f4f8; /* Light background on hover for unselected */
    color: #2d3748;
    box-shadow: 0 3px 8px rgba(0, 0, 0, 0.1);
}

/* Styling for tab content area */
.gradio-tabs > div[role="tabpanel"] {
    padding: 25px;
    background-color: #fdfdfe; /* Very light background for tab content */
    border-radius: 12px; /* Rounded corners for content area */
    border: 1px solid #e2e8f0;
    border-top: none; /* Remove top border if desired, or keep for separation */
}


/* Manager Tab Specific Styles */
.manager-tab-container {
    background-color: #f8faff; /* Very light blue background */
}
.manager-tab-container .worker-row {
    background-color: #e0f2fe; /* Lighter blue for worker rows */
    border-radius: 10px;
    padding: 18px;
    margin-bottom: 18px;
    border: 1px solid #bfdbfe;
    align-items: center;
    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.05); /* Subtle shadow for rows */
}
.manager-tab-container .worker-info-box label {
    color: #1e40af;
    font-weight: 600;
    margin-bottom: 5px; /* Space between label and input */
}
.manager-tab-container .worker-info-box input {
    background-color: #eff6ff; /* Even lighter blue for info boxes */
    border-radius: 8px;
    padding: 10px 14px;
    font-weight: 500;
    color: #1e40af;
    border: 1px solid #93c5fd !important; /* Subtle border */
    box-shadow: inset 0 1px 3px rgba(0,0,0,0.05); /* Inner shadow */
}
.manager-tab-container .task-input label {
    color: #4a5568;
    font-weight: 600;
    margin-bottom: 5px;
}
.manager-tab-container .task-input textarea {
    border-radius: 8px;
    border: 1px solid #a0aec0 !important;
    padding: 10px 14px;
    box-shadow: inset 0 1px 3px rgba(0,0,0,0.05);
    transition: border-color 0.2s ease;
}
.manager-tab-container .task-input textarea:focus {
    border-color: #3182ce !important; /* Highlight on focus */
}
.manager-tab-container .action-button {
    border-radius: 8px;
    padding: 12px 20px;
    font-weight: 600;
    cursor: pointer;
    transition: background-color 0.3s ease, transform 0.2s ease, box-shadow 0.3s ease;
    box-shadow: 0 4px 10px rgba(0,0,0,0.1); /* Stronger shadow */
    border: none !important;
    background-image: linear-gradient(to right, var(--button-start-color), var(--button-end-color)); /* Gradient */
    color: white;
}
.manager-tab-container .assign-button {
    --button-start-color: #22c55e;
    --button-end-color: #10b981;
}
.manager-tab-container .assign-button:hover {
    --button-start-color: #16a34a;
    --button-end-color: #059669;
    transform: translateY(-3px);
    box-shadow: 0 6px 15px rgba(34, 197, 94, 0.4);
}
.manager-tab-container .remove-button {
    --button-start-color: #ef4444;
    --button-end-color: #dc2626;
}
.manager-tab-container .remove-button:hover {
    --button-start-color: #dc2626;
    --button-end-color: #b91c1c;
    transform: translateY(-3px);
    box-shadow: 0 6px 15px rgba(239, 68, 68, 0.4);
}
.assignment-table {
    border: 1px solid #e2e8f0;
    border-radius: 10px;
    overflow: hidden;
    box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
}
.assignment-table table {
    width: 100%;
    border-collapse: collapse;
}
.assignment-table th, .assignment-table td {
    padding: 14px 18px;
    text-align: left;
    border-bottom: 1px solid #edf2f7;
}
.assignment-table th {
    background-color: #f7fafc;
    font-weight: 700;
    color: #4a5568;
    text-transform: uppercase;
    font-size: 0.9em;
}
.assignment-table tbody tr:nth-child(even) {
    background-color: #fbfdff; /* Zebra striping */
}

/* Search Tab Specific Styles */
.search-tab-container {
    background-color: #f8fdfb; /* Very light green background */
}
.search-dropdown {
    border-radius: 8px;
    border: 1px solid #6ee7b7 !important;
    box-shadow: 0 2px 5px rgba(0,0,0,0.05);
}

.search-container-row {
  display: flex !important;
  justify-content: center !important;
  align-items: center !important;
}

.search-container-inner {
  max-width: 400px;
  width: 100%;
}
.no-margin {
  margin-top: 0 !important;
  margin-bottom: 0 !important;
  padding-top: 0 !important;
}


.search-dropdown label {
    color: #059669;
    font-weight: 600;
}
.search-dropdown select {
    padding: 10px 14px;
    background-color: #ecfdf5 !important; /* Lighter green for dropdown background */
}
.search-button {
    background-color: #10b981;
    color: white;
    border-radius: 8px;
    padding: 12px 20px;
    font-weight: 600;
    transition: background-color 0.3s ease, transform 0.2s ease, box-shadow 0.3s ease;
    box-shadow: 0 4lyah 10px rgba(0,0,0,0.1);
    border: none !important;
    margin-top: 18px;
    background-image: linear-gradient(to right, #10b981, #065f46);
}
.search-button:hover {
    background-color: #059669;
    transform: translateY(-3px);
    box-shadow: 0 6px 15px rgba(16, 185, 129, 0.4);
}
.search-output {
    margin-top: 25px;
    padding: 20px;
    background-color: #ecfdf5;
    border: 1px solid #a7f3d0;
    border-radius: 10px;
    min-height: 120px;
    color: #065f46;
    box-shadow: inset 0 1px 5px rgba(0,0,0,0.08);
    line-height: 1.6;
}
.search-output a {
    color: #065f46;
    text-decoration: underline;
    font-weight: 500;
}

/* Statistics Tab Specific Styles */
.stats-tab-container {
    background-color: #fcf8ff; /* Very light purple background */
}
.sensor-type-selector .sensor-dropdown {
    border-radius: 8px;
    border: 1px solid #a78bfa !important;
    box-shadow: 0 2px 5px rgba(0,0,0,0.05);
}
.sensor-type-selector .sensor-dropdown label {
    color: #7c3aed;
    font-weight: 600;
}
.sensor-type-selector .sensor-dropdown select {
    padding: 10px 14px;
    background-color: #f3e8ff !important; /* Lighter purple for dropdown background */
}
.plots-grid {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(320px, 1fr)); /* Slightly larger min-width */
    gap: 25px; /* Increased gap */
    margin-top: 25px;
}
/* Ensure Plotly plots fit their containers */
.plotly-graph-div {
    width: 100% !important;
    height: 100% !important;
}
/* Hide the download button on the image */
.gradio-container .search-tab-container .gr-image > div[data-testid="image"] button {
  display: none !important;
}

/* Hide the label/filename below the image */
.gradio-container .search-tab-container .gr-image > label {
  display: none !important;
}
/* More aggressive attempt to hide the download button */
.gradio-container .gr-image button[aria-label="Download image"] {
    display: none !important;
}

/* Also hide any potential alternative download links */
.gradio-container .gr-image a {
    display: none !important;
}


/* Knowledge Hub tab */
    .knowledge-hub-container {
        background-color: #f0fff4; /* Very light green */
    }
    .knowledge-hub-container .tip-input label {
        color: #047857; /* Darker green for label */
        font-weight: 600;
    }
    .knowledge-hub-container .tip-input textarea {
        border-radius: 8px;
        border: 1px solid #34d399 !important; /* Green border */
        padding: 12px;
        box-shadow: inset 0 1px 3px rgba(0,0,0,0.05);
        transition: border-color 0.2s ease;
    }
     .knowledge-hub-container .tip-input textarea:focus {
        border-color: #059669 !important; /* Highlight on focus */
    }
    .knowledge-hub-container .add-tip-button {
        background-color: #10b981; /* Medium green */
        color: white;
        border-radius: 8px;
        padding: 12px 20px;
        font-weight: 600;
        transition: background-color 0.3s ease, transform 0.2s ease, box-shadow 0.3s ease;
        box-shadow: 0 4px 10px rgba(0,0,0,0.1);
        border: none !important;
        margin-top: 15px;
        background-image: linear-gradient(to right, #10b981, #059669);
    }
    .knowledge-hub-container .add-tip-button:hover {
        background-color: #04785e; /* Darker green on hover */
        transform: translateY(-3px);
        box-shadow: 0 6px 15px rgba(16, 185, 129, 0.4);
    }
     .knowledge-hub-container .tips-table table {
        width: 100%;
        border-collapse: collapse;
         border: 1px solid #a7f3d0; /* Green border for the table */
         border-radius: 10px;
         overflow: hidden; /* Ensures border-radius works */
         box-shadow: 0 4px 12px rgba(0, 0, 0, 0.05);
    }
     .knowledge-hub-container .tips-table th,
     .knowledge_hub-container .tips-table td {
        padding: 14px 18px;
        text-align: left;
        border-bottom: 1px solid #d1fae5; /* Lighter green border between rows */
     }
      .knowledge_hub-container .tips-table th {
        background-color: #ecfdf5; /* Very light green background for header */
        font-weight: 700;
        color: #065f46; /* Dark green text for header */
        text-transform: uppercase;
        font-size: 0.9em;
     }
     .knowledge_hub-container .tips-table tbody tr:nth-child(even) {
        background-color: #f0fff4; /* Alternate row color */
     }
      /* Ensure the DataFrame cell content wraps */
    .knowledge-hub-container .tips-table td {
        white-space: normal !important; /* Allow text to wrap */
    }


"""

# Launch the Gradio application
demo.launch()

Loaded assignments from Firebase: {'Alice Johnson': ['write code for machine A'], 'Bob Smith': ['fix machine B']}
[{'tip': 'dsadsad', 'uploader': 'ewewqe'}, {'tip': 'wqee', 'uploader': 'ewqe'}, {'tip': 'sdsd', 'uploader': 'ewqe'}, {'tip': 'fdefdf', 'uploader': 'dfd'}]
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4de41ca4213e31d544.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
